# Cell Profiler Pipeline Markdown
### Jindayi Liang


## Step One
#### Select pipeline, input folder and models
Select the folder, usually three parameter: **cp_pipeline** is the name of pipeline, the location of pipeline should be the same as this code. **cp_mudole** is the name of the trained model set of CellPose. **cp_mudole_dic** is the location of the model. the syntax should be similar to input directory, either <code style="background:grey;color:white">Default input Folder</code> or <code style="background:grey;color:white">Default Input Folder sub-folder</code> followed by subfolder of your module. 

In [ ]:

cp_pipline = "CS_0809.no.cp.v1.cppipe"
cp_mudole = "CP_testJIL_epoch500"
cp_mudole_dic = "Default Input Folder sub-folder|OneDrive - Johns Hopkins\\Cai_Lab\\16_CellProfilerPipeline\\230703-09_czi"

#### Select Input Files

Iput files should be located in <a>raw</a> folder in local path. Within raw folder, there should be the czi images. 

Run print(file_list) can show you the files in the folder. 

In [ ]:

from pathlib import Path
mydir = Path("raw/")
file_list = []
for file in mydir.iterdir():
    if file.suffix == '.czi':
        file_list.append(str(file)) 
#file_list

#### Split CZI files

<code style="background:grey;color:white">.czi</code> files are divided into tif files and store them in the <code style="background:grey;color:white">image folder</code>. The following chuck requires **aicsimageio** module in your python interpretor. Run this under <code style="background:grey;color:white">ImageAnalysis environment</code>. 

The loop are repeated based on number of positions (scenes), time stamps, and channels. Each loops would feed out a tif image with suffix at the end. If a image is at Scene3, Time2, channel2, then the suffix will be <code style="background:grey;color:white">S003.T002.C002</code>. 

You can customize your naming to meet the cellprofiler pipeline requirement. 

For example, 
If you have various three channel images and two channel images, you can change your naming in the loop: For images with 2 channels, name your images into C1C2; for images with 3 channels, name your images into C0C1C2. Eventually your pipeline can only look at blue channel and green channel.

In [ ]:
# convert czi to tiff
import os
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from pathlib import Path
if not os.path.exists('images'):
    os.makedirs('images')
for img in file_list: 
    filename = Path(img).stem
    img2 = AICSImage(img) # Open an czi image in file_list
    img_channels = img2.dims.C
    img_scenes = len(img2.scenes)
    img_time = img2.dims.T

    for S in range(img_scenes): #loop through scene by scene (aka positions)
        for T in range(img_time): #loop through time by time
            for C in range(img_channels):
                img2.set_scene(S)
                print(S,T,C)
                single_img_data = img2.get_image_data('ZYX',S=S,T=T,C=C)
                pos_scene = img2.current_scene
                #one channel image
                if img_channels == 1:
                    file_name = f'{filename}_{os.path.splitext(img)[1]}.S{S+1:03d}.T{T+1:03d}.{pos_scene}.ome.tif'
                    OmeTiffWriter.save (single_img_data,f'./images/{file_name}')
                #for two channel image, split C1C2
                #for three channel image, split C0C1C2
                if img_channels == 2:
                    file_name = f'{filename}_{os.path.splitext(img)[1]}.S{S+1:03d}.T{T+1:03d}.C{C+1:03d}.ome.tif'
                    OmeTiffWriter.save(single_img_data,f'./images/{file_name}')
                elif img_channels == 3:
                    file_name = f'{filename}_{os.path.splitext(img)[1]}.S{S:03d}.T{T:03d}.C{C:03d}.ome.tif'
                    OmeTiffWriter.save(single_img_data,f'./images/{file_name}')

## Step Two

Open CellProfiler in headless mode, started java environment, import example data and images, set empty object set, set empty measurement, load pipeline

In [ ]:
import cellprofiler_core.pipeline
import cellprofiler_core.preferences
import cellprofiler_core.utilities.java
import pathlib
cellprofiler_core.preferences.set_headless()
cellprofiler_core.utilities.java.start_java()
image_set_list = cellprofiler_core.image.ImageSetList()
image_set = image_set_list.get_image_set(0)
import skimage.data
x = skimage.data.camera()
image_x = cellprofiler_core.image.Image(x)
image_set.add("x", image_x)
skimage.io.imshow(image_set.get_image("x").pixel_data)
object_set = cellprofiler_core.object.ObjectSet()
objects  = cellprofiler_core.object.Objects()
object_set.add_objects(objects, "example")
measurements = cellprofiler_core.measurement.Measurements()
pipeline = cellprofiler_core.pipeline.Pipeline()
pipeline.load(cp_pipline)
pipeline.modules()

In [ ]:
import cellprofiler.modules.runcellpose
module = cellprofiler.modules.runcellpose.RunCellpose()
#determine where you put the module
module.module_num = 9
pipeline.add_module(module)
# pipeline.modules()[7] = pipeline.modules()[8]
pipeline.modules()

In [ ]:

pipeline.modules()[8].setting(1).set_value("cells")
pipeline.modules()[8].setting(2).set_value("200")
pipeline.modules()[8].setting(3).set_value("custom")
pipeline.modules()[8].setting(4).set_value("CellposeCell")
pipeline.modules()[8].setting(5).set_value("Yes")
pipeline.modules()[8].setting(6).set_value("Yes")
pipeline.modules()[8].setting(11).set_value(cp_mudole_dic)
pipeline.modules()[8].setting(12).set_value(cp_mudole)
pipeline.modules()[8].setting(21).set_value("No")
[print(setting.to_dict()) for setting in pipeline.modules()[8].settings()]





In [ ]:
[print(setting.to_dict()) for setting in pipeline.modules()[5].settings()]

In [ ]:
import os
folder_name = 'output'
if not os.path.isdir(folder_name):
    os.makedirs(folder_name)
current_dir = pathlib.Path().absolute()
cellprofiler_core.preferences.set_default_output_directory(f"output")
file_list = list(pathlib.Path('.').absolute().glob('images\\*.TIF'))
# print(file_list)
files = [file.as_uri() for file in file_list]
pipeline.read_file_list(files)
output_measurements = pipeline.run()

In [ ]:
output_measurements.get_measurement_columns()

In [ ]:

output_measurements.get_measurement('CellposeCell', 'Children_IdentifyPrimaryObjects_Count')